In [ ]:
try:
  df_platform
  print("Data already loaded")
except NameError:
  print("Loading data...")
  %run data.ipynb

In [ ]:
df_bank_reconciliation = df_banks.copy()
df_bank_reconciliation.head()

In [ ]:
df_platform_bank_related = df_platform[(df_platform['kind'] == 'CONTRIBUTION') | (df_platform['kind'] == 'EXPENSE') | (df_platform['kind'] == 'ADDED_FUNDS')]
print(f"{len(df_platform_bank_related)} bank related transactions out of {len(df_platform)} total transactions ({len(df_platform_bank_related)/len(df_platform)*100:.2f}%)")

In [ ]:
import pandas as pd
from utils import matchByDateAndValue
df_platform_bank_debits = matchByDateAndValue(df_platform_bank_related, ['createdAt', 'amountInHostCurrency.value'], df_bank_reconciliation, ['Post Date', 'Debit'] , '2 days', 0.05, True).dropna(subset=['df2key'])
df_platform_bank_credits = matchByDateAndValue(df_platform_bank_related, ['createdAt', 'amountInHostCurrency.value'], df_bank_reconciliation, ['Post Date', 'Credit'] , '2 days', 0.05, True).dropna(subset=['df2key'])
df_platform_bank = pd.concat([df_platform_bank_debits, df_platform_bank_credits])
print('Successfully merged {} rows out of {} from Value and Date'.format(len(df_platform_bank), len(df_bank_reconciliation)))

In [ ]:
df_left_bank = df_bank_reconciliation[~df_bank_reconciliation['index'].isin(df_platform_bank['index'])]
df_payouts_bank = matchByDateAndValue(df_stripe_payouts, ['automatic_payout_effective_at_utc', 'net'], df_left_bank, ['Post Date', 'Credit'] , '2 days', 0.05, True).dropna(subset=['df2key'])
df_left_bank = df_left_bank[~df_left_bank['index'].isin(df_payouts_bank['index'])]
print('Successfully merged {} rows out of {} from payouts Value and Date'.format(len(df_payouts_bank), len(df_left_bank)))